In [1]:
import os
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes.models import *
from langchain.text_splitter import RecursiveCharacterTextSplitter
from PyPDF2 import PdfReader
from azure.ai.formrecognizer import DocumentAnalysisClient
from openai import AzureOpenAI
from azure.search.documents.models import VectorizedQuery
import glob
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# 開発環境・本番環境でも同じ認証方式を使用するため、DefaultAzureCredentialを用いて認証情報を取得する。
# azure_credential = DefaultAzureCredential()
# token_provider = get_bearer_token_provider(
#     DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
# )

# 環境変数からAzure AI Search、Azure OpenAI、Azure Document Intelligenceのエンドポイントを取得する
search_service_endpoint = os.environ["SEARCH_SERVICE_ENDPOINT"]
aoai_endpoint = os.environ["AOAI_ENDPOINT"]
aoai_api_version = os.environ["AOAI_API_VERSION"]
document_intelligence_endpoint = os.environ["DOCUMENT_INTELLIGENCE_ENDPOINT"]

# Jupyter Notebook環境で手動で引数を設定
class Args:
    docs = "/workspaces/aoai-rag-handson/data/001018385.pdf"  # インデックス対象のファイルが格納されているディレクトリを指定
    chunksize = "1000"  # テキストを分割する際のサイズを指定
    overlap = "200"  # テキストを分割する際のオーバーラップサイズを指定
    remove = False  # インデックスを削除するかどうかを指定

args = Args()

# テキストを分割する際の区切り文字を指定する
separator = ["\n\n", "\n", "。", "、", " ", ""]

def create_index():
    """
    Azure AI Searchのインデックスを作成する
    """
    client = SearchIndexClient(endpoint= os.environ["SEARCH_SERVICE_ENDPOINT"], credential=AzureKeyCredential(os.environ["SEARCH_API_KEY"]))
    name = "docs3"

    # すでにインデックスが作成済みである場合には何もしない
    if 'docs3' in client.list_index_names():
        print("すでにインデックスが作成済みです")
        return

    # インデックスのフィールドを定義する
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="content", type="Edm.String", analyzer_name="ja.microsoft"),
        SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile")
    ]

    # セマンティック検索のための定義を行う
    semantic_settings = SemanticSearch(
        configurations=[
            SemanticConfiguration(
                name="default",
                prioritized_fields=SemanticPrioritizedFields(
                    title_field=None,
                    content_fields=[
                        SemanticField(field_name="content")
                    ],
                ),
            )
        ]
    )

    # ベクトル検索のための定義を行う
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw"
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw",
            )
        ]
    )

    # インデックスを作成する
    index = SearchIndex(name=name, fields=fields, vector_search=vector_search, semantic_search=semantic_settings)
    client.create_index(index)

def delete_index():
    """
    Azure AI Searchのインデックスを削除する
    """
    client = SearchIndexClient(endpoint= os.environ["SEARCH_SERVICE_ENDPOINT"], credential=AzureKeyCredential(os.environ["SEARCH_API_KEY"]))
    client.delete_index('docs3')

def index_docs(chunks: list):
    """
    ドキュメントをAzure AI Searchにインデックスする
    """
    # Azure AI SearchのAPIに接続するためのクライアントを生成する
    searchClient = SearchClient(
    endpoint= os.environ["SEARCH_SERVICE_ENDPOINT"],
    index_name="docs3",
    credential=AzureKeyCredential(os.environ["SEARCH_API_KEY"])
    )

    # Azure OpenAIのAPIに接続するためのクライアントを生成する
    openAIClient = AzureOpenAI(
    api_key = os.environ["AOAI_API_KEY"],  
    api_version = os.environ["AOAI_API_VERSION"],
    azure_endpoint = os.environ["AOAI_ENDPOINT"]
    )

    # チャンク化されたテキストとそのテキストのベクトルをAzure AI Searchにアップロードする
    for i, chunk in enumerate(chunks):
        print(f"{i+1}個目のチャンクを処理中...")
        print(chunk)
        response = openAIClient.embeddings.create(
            input = chunk,
            model = "text-embedding-3-small-deploy"
        )

        # チャンク化されたテキストとそのテキストのベクトルをAzure AI Searchにアップロードする
        document = {"id": str(i), "content": chunk, "contentVector": response.data[0].embedding}
        searchClient.upload_documents([document])

def create_chunk(content: str, separator: str, chunk_size: int = 512, overlap: int = 0):
    """
    テキストを指定したサイズで分割する
    """
    splitter = RecursiveCharacterTextSplitter(chunk_overlap=overlap, chunk_size=chunk_size, separators=separator)
    chunks = splitter.split_text(content)
    return chunks

# def extract_text_from_docs(filepath):
#     """
#     ドキュメントからテキストを抽出する
#     """
#     # Azure Document IntelligenceのAPIに接続するためのクライアントを生成する
#     form_recognizer_client = DocumentAnalysisClient(endpoint=document_intelligence_endpoint, credential=azure_credential)

#     # ドキュメントを読み込んで、Azure Document IntelligenceのAPIを呼び出して、テキストを抽出する
#     print(f"{filepath}内のテキストを抽出中...")
#     with open(filepath, "rb") as f:
#         poller = form_recognizer_client.begin_analyze_document("prebuilt-layout", document = f)
#     form_recognizer_results = poller.result()

#     # ドキュメントのテキストを抽出する
#     # ドキュメントのテキストは、ページごとに分割されているので、それを結合して返す
#     text = ""
#     for page in form_recognizer_results.pages:
#         for line in page.lines:
#             text += line.content
#     return text

def extract_text_from_docs(filepath):
    """
    PDFからテキストを抽出する
    """
    print(f"{filepath}内のテキストを抽出中...")
    with open(filepath, "rb") as f:
        reader = PdfReader(f)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def clean_text(text):
    """
    テキストから空白と特殊文字を取り除く
    """
    import re
    text = re.sub(r'\s+', ' ', text)  # 空白文字をスペースに置き換え
    text = re.sub(r'[^\w\s]', '', text)  # 特殊文字を除去
    return text

# Jupyter Notebook環境でのスクリプト実行部分
if args.remove:
    # 引数に--removeが指定されている場合には、インデックスを削除する
    delete_index()
else:
    # インデックスを作成する
    create_index()

    # 引数--docsで指定されたディレクトリ内のファイルを読み込んで、Azure AI Searchにインデックスする
    for filename in glob.glob(args.docs):
        # ドキュメントからテキストを抽出する
        content = extract_text_from_docs(filename)

        # テキストをクリーンアップする
        content = clean_text(content)

        # テキストを指定したサイズで分割する
        chunksize = int(args.chunksize)
        overlap = int(args.overlap)
        result = create_chunk(content, separator, chunksize, overlap)

        # テキストをAzure AI Searchにインデックスする
        index_docs(result)


/workspaces/aoai-rag-handson/data/001018385.pdf内のテキストを抽出中...
1個目のチャンクを処理中...
１ モデル就業規則 令和５年７月版 厚生労働省労働基準局監督課 ２ はじめに １ 就業規則の意義 労働者が安心して働ける明るい職場を作ることは事業規模や業種を問わずすべての 事業場にとって重要なことですそのためにはあらかじめ就業規則で労働時間や賃金を はじめ人事服務規律など労働者の労働条件や待遇の基準をはっきりと定め労使間 でトラブルが生じないようにしておくことが大切です ２ 就業規則の内容 就業規則に記載する事項には必ず記載しなければならない事項以下絶対的必要記 載事項といいますと各事業場内でルールを定める場合には記載しなければならない 事項以下相対的必要記載事項といいますがあります労働基準法昭和２２年法 律第４９号以下労基法といいます第８９条このほか使用者において任意に 記載し得る事項もあります 絶対的必要記載事項は次のとおりです １ 労働時間関係 始業及び終業の時刻休憩時間休日休暇並びに労働者を２組以上に分けて交 替に就業させる場合においては就業時転換に関する事項 ２ 賃金関係 賃金の決定計算及び支払の方法賃金の締切り及び支払の時期並びに昇給に関 する事項 ３ 退職関係 退職に関する事項解雇の事由を含みます 相対的必要記載事項は次のとおりです １ 退職手当関係 適用される労働者の範囲退職手当の決定計算及び支払の方法並びに退職手当 の支払の時期に関する事項 ２ 臨時の賃金最低賃金額関係 臨時の賃金等退職手当を除きます及び最低賃金額に関する事項 ３ 費用負担関係 ３ 労働者に食費作業用品その他の負担をさせることに関する事項 ４ 安全衛生関係 安全及び衛生に関する事項 ５ 職業訓練関係 職業訓練に関する事項 ６ 災害補償業務外の傷病扶助関係 災害補償及び業務外の傷病扶助に関する事項 ７ 表彰制裁関係 表彰及び制裁の種類及び程度に関する事項 ８ その他 事業場の労働者すべてに適用されるルールに関する事項 なお就業規則の内容は法令及び当該事業場において適用される労働協約に反しては なりません法令又は労働協約に反する就業規則については所轄労働基準監督署長はそ の変更を命ずることができます労基法第９２条 ３ 就業規則の作成